# Bilinear Pairings in Python, Solidity, and the EVM

Bilinear pairings allow us to take three numbers $a, b\ and\ c$ where $ab = c$, encrypt them to become $E(a), E(b), E(c)$ where $E$ is an encryption function, then send the two encrypted values to a verifier who can verify $E(a) E(b) = E(c)$ but not know the original values. 

However, for EC points and related usages, we express as follows:
$$P = pG$$
$$Q = qG$$
$$R = rG$$
Given that $pq = r$, we have a function $f$ such that
$$
f(P,Q) = f(R, G)
$$
where the $G$ means $1$ or that $P \times Q = R \times 1$

Usual notation used is $e(P, Q)$ where $P$ and $Q$ are points on EC and $e$ has nothing to do w/ natural log.

_Bilinear means that if a function takes two arguments, and one of them is held constant, and the other varies, then the output linearly varies with the non-constant argument._

## What is $e(P, Q)$ returning?

Scary output. It is a 12-dimensional array (or vector) The identity element is $(1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)$.

Let's say the output of $e(P, Q)$ is $G_T$ then it has all the properties of a cyclic group. I.e. it has closed binary operator, associative, it has inverse, it has a generator and it is homomorphic to another finite cyclic group in some way or other.

The binary operator of $G_T$ is roughly what we call "multiplication" so $8G_T = 2G_T \ast 4G_T$

## Symmetric and Asymmetric Groups

In practice, it turns out it is easier to create bilinear pairings when a different group (but same order) is different for both of the arguments. I.e.:
$$
e(a,b) \to c,\ a\ \epsilon\ G_1, b\ \epsilon\ G_2, c\ \epsilon\ G_3 $$ 
$$
e(aG_1, bG_2) = e(abG_1, G2) = e(G_1, abG_2)
$$

The group $G_T$ is no explicitly stated but is the codomain of e(G_1, G_2). One can think of $G_1$ and $G_2$ being different elliptic curves with different parameters but have **same number of points** and that would be valid because those are different groups.

In symmetric pairing we only use one EC. In practice, asymmetric pairing is used where we have different groups.

In `py_ecc` we use `from py_ecc.bn128 import G1, G2`.

## Field Extensions
We think of $x$ and $y$ as two dimensional points. _With_ field extensions, the $x$ and $y$ temselves become 2D objects as $(x,y)$. So, this is like EC points with more than two dimensions. An EC in $G_2$ is an EC where both $x$ and $y$ elements are two dimensional objects.

Etherium's $G_2$ uses EC with field extensions.

In [8]:
from py_ecc.bn128 import G1, G2, pairing, add, multiply, eq

print(G1)
print(G2)


(1, 2)
((10857046999023057135944570762232829481370756359578518086990519993285655852781, 11559732032986387107991004021392285783925812861821192530917403151452391805634), (8495653923123431417604973247489272438418190587263600148770280649306958101930, 4082367875863433681332203403145435568316851327593401208105741076214120093531))
The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [9]:
# Both G1 and G2 has same curve_order as per design. But the operations are same as we have in normal EC over finite field (i.e. G1)

print(eq(add(G1, G1), multiply(G1, 2)))
print(eq(add(G2, G2), multiply(G2, 2)))

# For convenience, this libray overrides +, *, ==, !=
print(G1 + G1 + G1 == 3*G1)

True
True
True


### Checking Pairings in python

In [6]:
# Remember we use asymmetric groups
P = multiply(G1, 2)
Q = multiply(G2, 4)
R = multiply(G1, 8)
R_dash = multiply(G2, 8)

print(eq(pairing(Q, P), pairing(G2, R)))
print(eq(pairing(Q, P), pairing(R_dash, G1)))

True
True


### Equality of products
Checking:
$$
e(P_2, P_1) \stackrel{?}{=} e(Q_2, Q_1)
$$

In [4]:
P_1 = multiply(G1, 3)
P_2 = multiply(G2, 8)

Q_1 = multiply(G1, 6)
Q_2 = multiply(G2, 4)

print(eq(pairing(P_2, P_1), pairing(Q_2, Q_1)))

True


### The Binary Operator of $G_T$

Elements in $G_T$ are combined using "multiplication" (or multiply) but that is just the syntactic override in python. In reality, the constants (or scalars) used to generat points ($P, Q, R$ etc.) behave as "powers" of a base. So,
$$P_1 = 2G_1\ and\ P_2 = 3G_2 $$
$$Q_1 = 3G_1\ and\ Q_2 = 4G_2$$
$$R_1 = 3G_1\ and\ R_2 = 6G_2$$
For the above, we can say that $e(P_2, P_1) \ast e(Q_2, Q_1) = e(R2, R1)$ i.e., (2\*3) + (3\*4) == 3\*6 which is 6 + 12 == 18. Here, even though we are "multiplying" the output of $e$, it behaves as if it was "power" of a constant base in $G_T$ and the scalars are "added". See below code for more details.

In [7]:
# 2 * 3 = 6
P_1 = multiply(G1, 2)
P_2 = multiply(G2, 3)

# 3 * 4 = 12
Q_1 = multiply(G1, 3)
Q_2 = multiply(G2, 4)

# 3 * 6 = 18
R_1 = multiply(G1, 3)
R_2 = multiply(G2, 6)

# b ^ {2 * 3} * b ^ {3 * 4} = b ^ {3 * 6}
# b ^ 6 * b ^ 12 = b ^ 18

print(eq(pairing(P_2, P_1) * pairing(Q_2, Q_1), pairing(R_2, R_1)))

True


# Bilinear Pairings in Ethereum

## EIP 197 Specification

`py_ecc` library is maintained by [Etherium Foundataion](https://ethereum.org/) and it is what powers the "precompile" at address $0x8$ in [PyEVM implementation](https://github.com/ethereum/py-evm). The Ethereum precompile defined in [EIP-197](https://eips.ethereum.org/EIPS/eip-197) works on points in $G_1$ and $G_2$, and implicitly works on points in $G_T$.

This specifications takes in a list of $G_1$ and $G_2$ laid out as follows:  
`A₁B₁A₂B₂...AₙBₙ : Aᵢ ∈ G1, Bᵢ ∈ G2`.  
Where  
A₁ = a₁G1  
B₁ = b₁G2  
A₂ = a₂G1  
B₂ = b₂G2  
...   
Aₙ = aₙG1  
Bₙ = bₙG2  
This precompile returns `1` if the following is `true`:  
`a₁b₁ + a₂b₂ + ... + aₙbₙ = 0` and `0` otherwise.  
We don't usually need the actual output of pairing, we just but only check the equality of pairings. Specifically in final step of [Groth16](https://www.rareskills.io/post/groth16) (the ZK algorithm used by tornoda cash) looks like:
$$ e(A_1, B_2) = e(\alpha_1, \beta_2) + e(L_1, \gamma_2) + e(C_1, \delta_2) $$  
Which is equivalent to:
$$ 0 = e(-A_1, B_2) + e(\alpha_1, \beta_2) + e(L_1, \gamma_2) + e(C_1, \delta_2) $$  
and we get -ve of a point by using `neg` function of `py_ecc.bn128`. Majority of ZK algorithms have such equations for equating pairings. 
![image](https://www.rareskills.io/wp-content/uploads/2024/08/935a00_63f7afa2360e49a09139ed2de90189fc~mv2.png)
The source code of tornodo cash indicates the same.  
Also, inside the Pairing function is where call to `address(8)` is done to complete pairing calculation and determine if the proof is valid or not. **WHAT DOES THIS MEAN MAN**.

NOTE: *Sometimes, the group is referred to as in the context of EIP 197.*

The key insight here is that if 
$$ab + cd = 0$$
Then it must also be true that
$$A_1 B_2 + C_1 D_2 = 0_12\ \ A_1, C_1\ \epsilon\ G_1\ and\ B_2, D_2\ \epsilon\ G_2$$
Here, the precompile is not "computing" the discrete logarithm, but it is simply checking if the sum is `0`.
Sum of pairing is `0` if an only if sum of products of discrete log is `0`.

# What is ZK-SNARK?

ZK-SNARK stands for **Zero-Knowledge Succinct Non-Interactive Argument of Knowledge**. It is a cryptographic proof system that allows a prover to convince a verifier that they know some information (e.g., a solution to a problem) without revealing the actual information or requiring any back-and-forth interaction.  

## Key Features of ZK-SNARKs:
1. **Zero-Knowledge:** The proof reveals nothing about the secret data, ensuring privacy.
2. **Succinct:** The proof is very small (a few hundred bytes) and can be verified quickly, regardless of the complexity of the computation.
3. **Non-Interactive:** The proof consists of a single message from the prover to the verifier, making it efficient and easy to use in distributed systems.
4. **Argument of Knowledge:** The prover must actually know the secret information, not just guess or simulate a proof.

# What is Groth16?
Groth16 is one of the most efficient implementations of ZK-SNARKs, proposed by *Jens Groth in 2016*. It improves the efficiency and simplicity of earlier ZK-SNARK constructions.

## Features of Groth16:
1. **Small Proof Size:** Proofs are only 192 bytes, regardless of the complexity of the computation.
2. **Fast Verification:** Verifying a proof involves a small number of cryptographic operations, making it highly efficient.
3. **Trusted Setup:** It requires a one-time trusted setup phase. The setup is specific to the computation being proven.

## How Groth16 Works (Simplified):
1. **Trusted Setup:** A secret "toxic waste" is generated and securely discarded. This ensures that no one can forge proofs.
2. **Prover:** Generates a proof by solving cryptographic equations that represent the problem's arithmetic circuit.
3. **Verifier:** Checks the proof against the public parameters and ensures the equations are satisfied.

# End to End Solidity Example of Bilinear Pairings

In [13]:
from py_ecc.bn128 import neg, multiply, G1, G2

a = 4
b = 3
c = 6
d = 2
print(neg(multiply(G1, a)))

print(multiply(G2, b))
print(multiply(G1, c))
print(multiply(G2, d))

(3010198690406615200373504922352659861758983907867017329644089018310584441462, 17861058253836152797273815394432013122766662423622084931972383889279925210507)
((2725019753478801796453339367788033689375851816420509565303521482350756874229, 7273165102799931111715871471550377909735733521218303035754523677688038059653), (2512659008974376214222774206987427162027254181373325676825515531566330959255, 957874124722006818841961785324909313781880061366718538693995380805373202866))
(4503322228978077916651710446042370109107355802721800704639343137502100212473, 6132642251294427119375180147349983541569387941788025780665104001559216576968)
((18029695676650738226693292988307914797657423701064905010927197838374790804409, 14583779054894525174450323658765874724019480979794335525732096752006891875705), (2140229616977736810657479771656733941598412651537078903776637920509952744750, 11474861747383700316476719153975578001603231366361248090558603872215261634898))


Now that we have the values of $a, b, c, d$ encrypted in $G_12$ anyone can use these values to prove that $ab + cd = 0$ by computing $e(A_1, B_2) + e(C_1, D_2) = 0$